In [1]:
# Dependencies and Setup
import csv 
from config import api_key
import pandas as pd
import numpy as np
import requests
import time
import json
import pymongo
from sqlalchemy import create_engine
import datetime as dt
url = "https://api.worldweatheronline.com/premium/v1/past-weather.ashx"
units = "metric"

In [2]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [3]:
csv_file = 'Resources/2018_airline_delay_causes.csv'
airline_data_df = pd.read_csv(csv_file, delimiter = ',')
airline_data_df.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2018,1,MQ,Envoy Air,BIS,"Bismarck/Mandan, ND: Bismarck Municipal",5.0,3.0,1.00,0.06,...,0.0,0.00,0.0,0.0,104.0,54.0,1.0,49.0,0.0,0.0
1,2018,1,MQ,Envoy Air,BNA,"Nashville, TN: Nashville International",110.0,21.0,7.17,1.16,...,0.0,5.92,3.0,0.0,897.0,344.0,37.0,226.0,0.0,290.0
2,2018,1,MQ,Envoy Air,BOI,"Boise, ID: Boise Air Terminal",32.0,8.0,0.22,0.35,...,0.0,1.82,0.0,0.0,353.0,9.0,18.0,233.0,0.0,93.0
3,2018,1,MQ,Envoy Air,BPT,"Beaumont/Port Arthur, TX: Jack Brooks Regional",63.0,11.0,1.75,1.08,...,0.0,5.19,3.0,0.0,657.0,83.0,34.0,130.0,0.0,410.0
4,2018,1,MQ,Envoy Air,BUF,"Buffalo, NY: Buffalo Niagara International",31.0,12.0,0.82,3.00,...,0.0,1.55,0.0,0.0,484.0,27.0,136.0,207.0,0.0,114.0


In [4]:
airline_data_df.columns

Index(['year', ' month', 'carrier', 'carrier_name', 'airport', 'airport_name',
       'arr_flights', 'arr_del15', 'carrier_ct', ' weather_ct', 'nas_ct',
       'security_ct', 'late_aircraft_ct', 'arr_cancelled', 'arr_diverted',
       ' arr_delay', ' carrier_delay', 'weather_delay', 'nas_delay',
       'security_delay', 'late_aircraft_delay'],
      dtype='object')

In [5]:
new_airline_data_df = airline_data_df[['year',' month','carrier','carrier_name','airport','airport_name','arr_flights','arr_cancelled', ' arr_delay', ' carrier_delay','weather_delay']].copy()

new_airline_data_df.head(10)

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_cancelled,arr_delay,carrier_delay,weather_delay
0,2018,1,MQ,Envoy Air,BIS,"Bismarck/Mandan, ND: Bismarck Municipal",5.0,0.0,104.0,54.0,1.0
1,2018,1,MQ,Envoy Air,BNA,"Nashville, TN: Nashville International",110.0,3.0,897.0,344.0,37.0
2,2018,1,MQ,Envoy Air,BOI,"Boise, ID: Boise Air Terminal",32.0,0.0,353.0,9.0,18.0
3,2018,1,MQ,Envoy Air,BPT,"Beaumont/Port Arthur, TX: Jack Brooks Regional",63.0,3.0,657.0,83.0,34.0
4,2018,1,MQ,Envoy Air,BUF,"Buffalo, NY: Buffalo Niagara International",31.0,0.0,484.0,27.0,136.0
5,2018,1,MQ,Envoy Air,BWI,"Baltimore, MD: Baltimore/Washington Internatio...",112.0,3.0,2198.0,567.0,101.0
6,2018,1,MQ,Envoy Air,BZN,"Bozeman, MT: Bozeman Yellowstone International",36.0,0.0,227.0,43.0,27.0
7,2018,1,MQ,Envoy Air,CAE,"Columbia, SC: Columbia Metropolitan",64.0,1.0,611.0,120.0,150.0
8,2018,1,MQ,Envoy Air,CHO,"Charlottesville, VA: Charlottesville Albemarle",53.0,3.0,696.0,123.0,6.0
9,2018,1,MQ,Envoy Air,CHS,"Charleston, SC: Charleston AFB/International",63.0,12.0,291.0,108.0,1.0


In [6]:
new_airline_data_df = new_airline_data_df.rename(columns={' month':'month'})
new_airline_data_df.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_cancelled,arr_delay,carrier_delay,weather_delay
0,2018,1,MQ,Envoy Air,BIS,"Bismarck/Mandan, ND: Bismarck Municipal",5.0,0.0,104.0,54.0,1.0
1,2018,1,MQ,Envoy Air,BNA,"Nashville, TN: Nashville International",110.0,3.0,897.0,344.0,37.0
2,2018,1,MQ,Envoy Air,BOI,"Boise, ID: Boise Air Terminal",32.0,0.0,353.0,9.0,18.0
3,2018,1,MQ,Envoy Air,BPT,"Beaumont/Port Arthur, TX: Jack Brooks Regional",63.0,3.0,657.0,83.0,34.0
4,2018,1,MQ,Envoy Air,BUF,"Buffalo, NY: Buffalo Niagara International",31.0,0.0,484.0,27.0,136.0


In [7]:
new_airline_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20231 entries, 0 to 20230
Data columns (total 11 columns):
year              20231 non-null int64
month             20231 non-null int64
carrier           20231 non-null object
carrier_name      20231 non-null object
airport           20231 non-null object
airport_name      20231 non-null object
arr_flights       20214 non-null float64
arr_cancelled     20214 non-null float64
 arr_delay        20214 non-null float64
 carrier_delay    20214 non-null float64
weather_delay     20214 non-null float64
dtypes: float64(5), int64(2), object(4)
memory usage: 1.7+ MB


In [8]:
new_airline_data_df[['City','State/Airport']] = new_airline_data_df.airport_name.str.split(expand=True,pat= ':')
new_airline_data_df.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_cancelled,arr_delay,carrier_delay,weather_delay,City,State/Airport
0,2018,1,MQ,Envoy Air,BIS,"Bismarck/Mandan, ND: Bismarck Municipal",5.0,0.0,104.0,54.0,1.0,"Bismarck/Mandan, ND",Bismarck Municipal
1,2018,1,MQ,Envoy Air,BNA,"Nashville, TN: Nashville International",110.0,3.0,897.0,344.0,37.0,"Nashville, TN",Nashville International
2,2018,1,MQ,Envoy Air,BOI,"Boise, ID: Boise Air Terminal",32.0,0.0,353.0,9.0,18.0,"Boise, ID",Boise Air Terminal
3,2018,1,MQ,Envoy Air,BPT,"Beaumont/Port Arthur, TX: Jack Brooks Regional",63.0,3.0,657.0,83.0,34.0,"Beaumont/Port Arthur, TX",Jack Brooks Regional
4,2018,1,MQ,Envoy Air,BUF,"Buffalo, NY: Buffalo Niagara International",31.0,0.0,484.0,27.0,136.0,"Buffalo, NY",Buffalo Niagara International


In [9]:
new_airline_data_df[['city','State']] = new_airline_data_df['City'].str.split(expand=True,pat= ',')
new_airline_data_df.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_cancelled,arr_delay,carrier_delay,weather_delay,City,State/Airport,city,State
0,2018,1,MQ,Envoy Air,BIS,"Bismarck/Mandan, ND: Bismarck Municipal",5.0,0.0,104.0,54.0,1.0,"Bismarck/Mandan, ND",Bismarck Municipal,Bismarck/Mandan,ND
1,2018,1,MQ,Envoy Air,BNA,"Nashville, TN: Nashville International",110.0,3.0,897.0,344.0,37.0,"Nashville, TN",Nashville International,Nashville,TN
2,2018,1,MQ,Envoy Air,BOI,"Boise, ID: Boise Air Terminal",32.0,0.0,353.0,9.0,18.0,"Boise, ID",Boise Air Terminal,Boise,ID
3,2018,1,MQ,Envoy Air,BPT,"Beaumont/Port Arthur, TX: Jack Brooks Regional",63.0,3.0,657.0,83.0,34.0,"Beaumont/Port Arthur, TX",Jack Brooks Regional,Beaumont/Port Arthur,TX
4,2018,1,MQ,Envoy Air,BUF,"Buffalo, NY: Buffalo Niagara International",31.0,0.0,484.0,27.0,136.0,"Buffalo, NY",Buffalo Niagara International,Buffalo,NY


In [ ]:
new_airline_data_df['city'] = new_airline_data_df['city'].str.split(expand=True,pat= '/')
new_airline_data_df.head()`

In [ ]:
final_airline_df = new_airline_data_df.drop(['airport_name','State/Airport'], axis=1)
final_airline_df.head()

In [ ]:
popular_airline = ['LAX','ATL','ORD','DEN','DFW']
top_airline = final_airline_df.loc[final_airline_df['airport'].isin(popular_airline)]
top_airline.head(10)

In [ ]:
final_df = top_airline.sort_values('month')
final_df.head(10)
final_df.to_csv('Resources/final_airline_data.csv')

In [ ]:
year = top_airline.groupby(['year','month','city'], as_index=False).mean()
year.head(10)

In [ ]:
def month_json_to_df(city_name):
    start_date=["2018-01-01","2018-02-01","2018-03-01","2018-04-01","2018-05-01","2018-06-01","2018-07-01","2018-08-01","2018-09-01","2018-10-01","2018-11-01","2018-12-01"]
    end_date=["2018-01-31","2018-02-28","2018-03-31","2018-04-30","2018-05-31","2018-06-30","2018-07-31","2018-08-31","2018-09-30","2018-10-31","2018-11-30","2018-12-31"]
    for i in range(12):
        query_url = f"{url}?key={api_key}&q={city_name}&format=json&extra=2018-01-01&date={start_date[i]}&enddate={end_date[i]}"
        response=requests.get(query_url)
        json_file = response.json()
        city = json_file['data']['request'][0]['query']
    
        date=[]
        weather_desc=[]
        for weather_row in json_file['data']['weather']:
            date.append(weather_row['date'])
            weather_desc.append(weather_row['hourly'][0]['weatherDesc'][0]['value'])
        if i ==0:
            final_DF=pd.DataFrame({'city':city, 'date':date, 'weather':weather_desc})
        else:
            temp_df=pd.DataFrame({'city':city, 'date':date, 'weather':weather_desc})
            final_DF=pd.concat([final_DF, temp_df])
    return final_DF

In [ ]:
lax_df=month_json_to_df('Los Angeles')
lax_df.to_csv('Resources/lax_df.csv')
#lax_df = lax_df.rename(columns= {"city":"city_name", "date":"date_of","weather":"weather_desc"})
lax_df.head()

In [ ]:
chi_df=month_json_to_df("Chicago")
chi_df.head()

In [ ]:
dfw_df=month_json_to_df("Dallas")
dfw_df.head()

In [ ]:
atl_df=month_json_to_df("Atlanta")
atl_df.head()

In [ ]:
den_df=month_json_to_df("Denver")
den_df.head()

In [ ]:
final = pd.concat([lax_df, dfw_df, chi_df, atl_df, den_df])
final_weather = final


In [ ]:
final_weather.weather.unique()

In [ ]:
final['weather_val']=final['weather'].replace({'Clear':15, 'Overcast':14,'Partly cloudy':13, 'Cloudy':12, 'Fog':11,\
                          'Freezing fog':10, 'Mist':9, 'Patchy light drizzle':8, 'Light drizzle':7, 'Patchy rain possible':6,\
                         'Light rain':5, 'Light rain shower':4, 'Moderate rain at times':3,'Moderate rain':2, 'Moderate or heavy rain shower':1,\
                         'Thundery outbreaks possible':0})


In [ ]:
final.info()

In [ ]:
final['date']=pd.to_datetime(final['date'])

In [ ]:
final.info()

In [ ]:
final['month']= final['date'].dt.month

In [ ]:
final['year']= final['date'].dt.year

In [ ]:
final.head()

In [ ]:
final.columns

In [ ]:
final.info()

In [ ]:
final[['city','country']] = final.city.str.split(expand= True,pat=',')

In [ ]:
final.head()

In [ ]:
final_df = final.groupby(['year','month','city'],as_index=False).mean()#.agg(lambda x:x.value_counts().index[0])

final_df.weather_val = np.round(final_df.weather_val)

In [ ]:
final_df

In [ ]:
final_df[final_df.city=='Los Angeles']

In [ ]:
final_df.weather_val.unique()

In [ ]:
final_df.weather_val.value_counts()

In [ ]:
city_list = final.city.unique()

In [ ]:
for city in city_list:
    print(city)
#     print(final[(final.city==city)&(final.month == 5)]['weather'].value_counts())
    print(final[(final.city==city)]['weather'].value_counts())

    print('-----------------------')

In [ ]:
final_df.head(10) 
#year.head()

In [ ]:
year.head(10)

In [ ]:
final_df = final_df.merge(year,on=['year','month','city'])

In [ ]:
final_df.head(10)

In [ ]:
#final_df['year'] = pd.to_date(final_df['year'])
#final_df.head(5)

In [ ]:
# date_stngs = ('2008-12-20','2008-12-21','2008-12-22','2008-12-23')
# a = pd.Series([pd.to_datetime(date) for date in date_stngs])
#df['col'] = pd.to_datetime(df['col'])

In [ ]:
final_df.to_csv('Resources/final_df.csv',index=False)

In [ ]:
year.to_csv('Resources/year.csv',index=False)